In [1]:
from __future__ import print_function

import numpy as np
import tensorflow as tf

# Import MNIST data
import load_data as data

/Library/Frameworks/Python.framework/Versions/3.6/lib/python3.6/importlib/_bootstrap.py:219: RuntimeWarning: compiletime version 3.5 of module 'tensorflow.python.framework.fast_tensor_util' does not match runtime version 3.6
  return f(*args, **kwds)


In [2]:
# n different points we want to cluster
# We have 59 points in all 3 categories * 2 people * 10 repetitions (maia_fall only had 9)
train_trials = 59
test_trials = 16
# Tensor length of 80 since that's how many frames we decided on--easily modifiable
NUM_FRAMES = 80
NUM_FEATURES = 36

# Starting off with just one point so that it's easier to follow along with tutorials
# Will switch to more than one point later
# joint_index specifies which of the 36 joints we want, 0 is nose
# Find corresponding values at https://github.com/CMU-Perceptual-Computing-Lab/openpose/blob/master/doc/output.md
joint_index = 0 

Xtr, Ytr = data.train_get_three_dimension(train_trials)
Xte, Yte = data.test_get_three_dimension(test_trials)

# Xtr, Ytr = data.get_one_keypoint(n, joint_index)
# Xte, Yte = data.get_one_keypoint(n, 2)
# Xtr, Ytr = mnist.train.next_batch(5000) #5000 for training (nn candidates)
# Xte, Yte = mnist.test.next_batch(200) #200 for testing

# tf Graph Input
xtr = tf.placeholder("float", [train_trials, NUM_FRAMES, NUM_FEATURES])
xte = tf.placeholder("float", [NUM_FRAMES, NUM_FEATURES])

# Nearest Neighbor calculation using L1 Distance
# Calculate L1 Distance
distance = tf.reduce_sum(tf.abs(tf.add(xtr, tf.negative(xte))), reduction_indices=1)
# Prediction: Get min distance index (Nearest neighbor)
pred = tf.argmin(distance, 0)

# Initialize the variables (i.e. assign their default value)
init = tf.global_variables_initializer()

Instructions for updating:
Use `argmin` instead


In [4]:
# Start training
with tf.Session() as sess:
    accuracy = 0.
    # Run the initializer
    sess.run(init)

    # loop over test data
    for i in range(len(Xte)):
        # Get nearest neighbor
        nn_index = sess.run(pred, feed_dict={xtr: Xtr, xte: Xte[i, :, :]})
        # Get nearest neighbor class label and compare it to its true label
        print("Test", i, "Prediction:", np.argmax(Ytr[nn_index]) + 1, \
            "True Class:", np.argmax(Yte[i]) + 1)
        # Calculate accuracy
        if np.argmax(Ytr[nn_index]) == np.argmax(Yte[i]):
            accuracy += 1./len(Xte)
            print(i)
    print("Done!")
    print("Accuracy:", accuracy)

Test 0 Prediction: 3 True Class: 1
Test 1 Prediction: 3 True Class: 2
Test 2 Prediction: 2 True Class: 3
Test 3 Prediction: 2 True Class: 2
3
Test 4 Prediction: 3 True Class: 2
Test 5 Prediction: 3 True Class: 3
5
Test 6 Prediction: 2 True Class: 1
Test 7 Prediction: 1 True Class: 1
7
Test 8 Prediction: 2 True Class: 2
8
Test 9 Prediction: 2 True Class: 2
9
Test 10 Prediction: 2 True Class: 1
Test 11 Prediction: 3 True Class: 3
11
Test 12 Prediction: 2 True Class: 2
12
Test 13 Prediction: 3 True Class: 3
13
Test 14 Prediction: 1 True Class: 1
14
Test 15 Prediction: 3 True Class: 1
Done!
Accuracy: 0.5625
